# 🧠 Hybrid Brain Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

### Combining Frequency-Domain and Spatial-Domain Analysis

**Our Approach:**
1. **FFT-Based Detection** - Analyzes texture in frequency domain
2. **Blob Detection** - Identifies bright, compact internal masses
3. **Ensemble Voting** - Combines methods for robust detection

**Key Innovation:**
- Filters skull and peripheral structures
- Focuses on internal brain masses
- Requires compact, isolated regions (not scattered bright pixels)

In [ ]:
# ========== SETUP ==========
!pip install -q numpy matplotlib scipy scikit-image pandas

!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py

!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, disk, binary_erosion
from skimage.segmentation import clear_border

def load_image_safe(filepath):
    img = PILImage.open(filepath).convert('L')
    return np.array(img).astype(np.float64) / 255.0

def load_mask_safe(filepath):
    img = PILImage.open(filepath).convert('L')
    return (np.array(img) > 127).astype(np.uint8)

def remove_skull_and_borders(mask, image, border_fraction=0.15):
    """
    Remove skull and peripheral structures.
    Tumors are INTERNAL - they don't touch image edges.
    """
    h, w = mask.shape
    
    # Create an internal region mask (exclude outer border)
    border_h = int(h * border_fraction)
    border_w = int(w * border_fraction)
    
    internal_region = np.zeros_like(mask, dtype=bool)
    internal_region[border_h:-border_h, border_w:-border_w] = True
    
    # Only keep regions that are mostly within internal area
    labeled = label(mask)
    cleaned = np.zeros_like(mask, dtype=bool)
    
    for region in regionprops(labeled):
        region_pixels = (labeled == region.label)
        internal_pixels = np.logical_and(region_pixels, internal_region).sum()
        
        # Region must be >70% internal (not on edges)
        if internal_pixels / region.area > 0.7:
            cleaned[region_pixels] = True
    
    return cleaned

# ========== METHOD 1: FFT-BASED DETECTION ==========
def fft_tumor_detection(image, sensitivity=0.5):
    """
    Frequency-domain tumor detection.
    """
    h, w = image.shape
    mean_val = image.mean()
    std_val = image.std()
    
    results = {}
    
    # High-pass
    try:
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=30)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        
        hp_thresh = np.percentile(hp_img, 90 - sensitivity * 5)  # Stricter
        hp_mask = hp_img > hp_thresh
        
        # Must be bright in original image
        hp_mask = np.logical_and(hp_mask, image > mean_val + std_val)
        hp_mask = binary_closing(hp_mask, disk(2))
        hp_mask = remove_small_objects(hp_mask, min_size=100)
        hp_mask = remove_skull_and_borders(hp_mask, image)
        
        results['FFT_HighPass'] = hp_mask
    except:
        results['FFT_HighPass'] = np.zeros_like(image, dtype=bool)
    
    # Band-pass
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        
        bp_thresh = np.percentile(bp_img, 85 - sensitivity * 5)  # Stricter
        bp_mask = bp_img > bp_thresh
        
        bp_mask = np.logical_and(bp_mask, image > mean_val + std_val)
        bp_mask = binary_closing(bp_mask, disk(2))
        bp_mask = remove_small_objects(bp_mask, min_size=100)
        bp_mask = remove_skull_and_borders(bp_mask, image)
        
        results['FFT_BandPass'] = bp_mask
    except:
        results['FFT_BandPass'] = np.zeros_like(image, dtype=bool)
    
    # Combined FFT
    fft_combined = np.logical_and(results['FFT_HighPass'], results['FFT_BandPass'])  # Both must agree!
    results['FFT_Combined'] = fft_combined
    
    return results

# ========== METHOD 2: BLOB DETECTION ==========
def blob_tumor_detection(image, sensitivity=0.5):
    """
    Spatial blob detection for compact, internal bright masses.
    """
    h, w = image.shape
    mean_val = image.mean()
    std_val = image.std()
    
    # VERY bright threshold - tumors are much brighter than brain
    bright_threshold = mean_val + (2.5 - sensitivity * 0.5) * std_val  # Stricter!
    bright_mask = image > bright_threshold
    
    # Also top percentile
    percentile_threshold = np.percentile(image, 95 - sensitivity * 3)  # Stricter!
    percentile_mask = image > percentile_threshold
    
    # Both must agree
    blob_mask = np.logical_and(bright_mask, percentile_mask)
    
    # Clean up
    blob_mask = binary_opening(blob_mask, disk(2))
    blob_mask = binary_closing(blob_mask, disk(3))
    blob_mask = remove_small_objects(blob_mask, min_size=150)  # Larger min size
    
    # Remove skull/borders
    blob_mask = remove_skull_and_borders(blob_mask, image, border_fraction=0.15)
    
    # Filter by shape and compactness
    labeled = label(blob_mask)
    filtered = np.zeros_like(blob_mask)
    
    for region in regionprops(labeled, intensity_image=image):
        # Size: 200 to 30% of image
        if region.area < 200 or region.area > h * w * 0.3:
            continue
        
        # Shape: must be reasonably compact
        circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
        if circularity < 0.15:  # Not too irregular
            continue
        
        # Solidity: must be solid, not ring-shaped
        if region.solidity < 0.7:
            continue
        
        # Intensity: must be MUCH brighter than background
        region_intensity = region.mean_intensity
        if region_intensity < mean_val + 1.5 * std_val:
            continue
        
        filtered[labeled == region.label] = True
    
    return {'Blob_Detection': filtered}

# ========== METHOD 3: HYBRID ENSEMBLE ==========
def hybrid_tumor_detection(image, sensitivity=0.5):
    """
    Hybrid detection with STRICT voting.
    Requires BOTH FFT and Blob to agree for high confidence.
    """
    fft_results = fft_tumor_detection(image, sensitivity)
    blob_results = blob_tumor_detection(image, sensitivity)
    
    all_results = {**fft_results, **blob_results}
    
    # STRICT voting: need both FFT AND Blob to detect something
    vote_map = (fft_results['FFT_Combined'].astype(float) * 2.0 +  # FFT gets 2 votes
                blob_results['Blob_Detection'].astype(float) * 1.5)  # Blob gets 1.5 votes
    
    # Need at least 3 votes (BOTH methods must agree!)
    combined_mask = vote_map >= (3.0 - sensitivity * 0.5)
    
    # Final cleanup
    combined_mask = binary_closing(combined_mask, disk(2))
    combined_mask = remove_small_objects(combined_mask, min_size=200)
    
    # Extra filtering: must be compact and internal
    labeled = label(combined_mask)
    final_mask = np.zeros_like(combined_mask)
    
    for region in regionprops(labeled):
        # Must be compact
        if region.eccentricity > 0.95:  # Too elongated
            continue
        if region.solidity < 0.7:  # Not solid enough
            continue
        
        final_mask[labeled == region.label] = True
    
    all_results['Hybrid_Combined'] = final_mask
    
    return all_results

def analyze_detection(image, mask):
    """
    Analyze with strict criteria.
    """
    h, w = image.shape
    total_pixels = h * w
    tumor_pixels = mask.sum()
    tumor_area_pct = (tumor_pixels / total_pixels) * 100
    
    # Minimum size: 0.5% of image
    if tumor_pixels < total_pixels * 0.005:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    coords = np.where(mask)
    if len(coords[0]) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    center = (int(np.mean(coords[0])), int(np.mean(coords[1])))
    
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    largest = max(regions, key=lambda x: x.area)
    
    # Confidence
    # Size: 0.5% - 10% is good
    if 0.5 <= tumor_area_pct <= 10:
        size_score = 1.0
    elif tumor_area_pct < 0.5:
        size_score = tumor_area_pct / 0.5
    else:
        size_score = max(0, 1 - (tumor_area_pct - 10) / 15)
    
    # Shape
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2, 1.0)
    
    # Intensity
    tumor_intensity = image[mask].mean()
    bg_intensity = image[~mask].mean() if (~mask).any() else 0
    contrast = tumor_intensity - bg_intensity
    contrast_score = min(max(contrast * 4, 0), 1.0)
    
    confidence = (size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4)
    
    # Higher minimum threshold
    if confidence < 0.35:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    return {
        'detected': True,
        'area_percent': tumor_area_pct,
        'center': center,
        'confidence': np.clip(confidence, 0, 1),
        'num_regions': len(regions)
    }

image = None
mask = None
print('✅ Hybrid Brain Tumor Detection System Ready!')
print('   Enhanced skull filtering + strict voting')
print('   Focuses on compact, internal masses only')

---
## 📁 Load Brain MRI

In [ ]:
# ========== UPLOAD ==========
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    image = load_image_safe(filename)
    mask = None
    
    print(f'✅ Loaded: {filename}')
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    plt.title('Brain MRI')
    plt.axis('off')
    plt.show()
else:
    print('❌ No file')

---
## 🔬 FFT Analysis

In [ ]:
if image is not None:
    F_shift, mag = ts.compute_fft_spectrum(image)
    fig = ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()
else:
    print('⚠️ No image')

---
## 🎯 Hybrid Detection

In [ ]:
if image is not None:
    print('Running detection...\n')
    
    results = hybrid_tumor_detection(image, sensitivity=0.5)
    methods = ['FFT_HighPass', 'FFT_BandPass', 'FFT_Combined', 'Blob_Detection', 'Hybrid_Combined']
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 11))
    axes = axes.ravel()
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontsize=13, fontweight='bold')
    axes[0].axis('off')
    
    for idx, method in enumerate(methods, 1):
        detected_mask = results[method]
        analysis = analyze_detection(image, detected_mask)
        
        axes[idx].imshow(image, cmap='gray')
        axes[idx].imshow(detected_mask, cmap='Reds', alpha=0.5)
        
        if analysis['center']:
            row, col = analysis['center']
            axes[idx].plot(col, row, 'g+', markersize=20, markeredgewidth=3)
        
        status = f"DETECTED\n{analysis['area_percent']:.1f}%" if analysis['detected'] else "Not detected"
        axes[idx].set_title(f"{method}\n{status}", fontsize=11, fontweight='bold')
        axes[idx].axis('off')
    
    plt.suptitle('Hybrid Detection (Skull-Filtered)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary
    print('\n' + '='*70)
    print(f'{"Method":<20} {"Detected":<12} {"Area %":<12}')
    print('='*70)
    for method in methods:
        analysis = analyze_detection(image, results[method])
        det = '✓ YES' if analysis['detected'] else '✗ No'
        area = f"{analysis['area_percent']:.2f}%" if analysis['detected'] else '-'
        print(f'{method:<20} {det:<12} {area:<12}')
    print('='*70)
else:
    print('⚠️ No image')

---
## 📋 FINAL VERDICT

In [ ]:
if image is not None and 'results' in dir():
    final_mask = results['Hybrid_Combined']
    final_analysis = analyze_detection(image, final_mask)
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    if final_analysis['detected']:
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.5)
        if final_analysis['center']:
            row, col = final_analysis['center']
            axes[1].plot(col, row, 'g+', markersize=30, markeredgewidth=4)
    axes[1].set_title('Detection', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].axis('off')
    
    if final_analysis['detected']:
        if final_analysis['confidence'] > 0.7:
            status = '⚠️ TUMOR DETECTED'
            verdict = '🔴 HIGH CONFIDENCE'
        elif final_analysis['confidence'] > 0.45:
            status = '❓ POSSIBLE TUMOR'
            verdict = '🟡 MEDIUM CONFIDENCE'
        else:
            status = '❓ SUSPICIOUS'
            verdict = '🟠 LOW CONFIDENCE'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Normal brain'
    
    results_text = f"""
    
    {status}
    
    ──────────────────────────
    
    Confidence: {final_analysis['confidence']*100:.1f}%
    
    Area: {final_analysis['area_percent']:.2f}%
    
    Method: Hybrid Ensemble
    (FFT + Blob voting)
    
    ──────────────────────────
    
    {verdict}
    """
    
    axes[2].text(0.05, 0.5, results_text, fontsize=13, verticalalignment='center',
                fontfamily='monospace', 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[2].set_title('Diagnosis', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*60)
    print(f'  {verdict}')
    print('='*60)
else:
    print('⚠️ Run detection first!')